In [ ]:
import paho.mqtt.client as mqtt
import statistics as st
import json
import ast
import re

SERVER = '147.228.124.230'  # RPi
TOPIC = 'ite/#'
#list dat i.e. blue = [[list_teplot], posledni, prumer, median, nejvyssi, nejnizsi, ]
list_blueTmprs = []
list_greenTmprs = []
list_pinkTmprs = []
list_redTmprs = []
list_blackTmprs = []
list_orangeTmprs = []
list_yellowTmprs = []

list_blueStat = []
list_greenStat = []
list_pinkStat = []
list_redStat = []
list_blackStat = []
list_orangeStat = []
list_yellowStat = []

def statistics_(list):
    if len(list) >= 1:
        list_stat =  [st.fmean(list), st.median(list), max(list), min(list), list[len(list)-1]]
        lis_statRound = [round(x,3) for x in list_stat]
        return lis_statRound
    else:
        return ['No data','No data','No data','No data','No data']
    
# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, mid, qos):
    print('Connected with result code qos:', str(qos))
    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect then subscriptions will be renewed.
    client.subscribe(TOPIC)

# The callback for when a PUBLISH message is received from the server.
def on_message(client, userdata, msg):
    
    global list_blueTmprs, list_greenTmprs, list_pinkTmprs, list_redTmprs, list_blackTmprs, list_orangeTmprs, list_yellowTmprs
    global list_blueStat, list_greenStat, list_pinkStat, list_redStat, list_blackStat, list_orangeStat, list_yellowStat
    global time_blue, time_black, time_pink, time_red, time_orange, time_yellow
    if (msg.payload == 'Q'):
        client.disconnect()
    print(msg.payload)
    try:
        dict_msg_str = str(msg.payload.decode("utf-8","ignore"))
        dict_msg_rep = dict_msg_str.replace("'", '"')
        dict_msg = json.loads(dict_msg_rep)  # vraci slovnik, parametr je json file
        if (team(dict_msg) == "blue"):
            list_blueTmprs.append(teplota(dict_msg))
            time_blue = cas(dict_msg)
        
        if (team(dict_msg) == "black"):
            list_blackTmprs.append(teplota(dict_msg))
            time_black = cas(dict_msg)
            
        
        if (team(dict_msg) == "pink"):
            list_pinkTmprs.append(teplota(dict_msg))
            time_pink = cas(dict_msg)
            
        
        if (team(dict_msg) == "red"):
            list_redTmprs.append(teplota(dict_msg))
            time_red = cas(dict_msg)
            
        
        if (team(dict_msg) == "orange"):
            list_orangeTmprs.append(teplota(dict_msg))
            time_orange = cas(dict_msg)
            
        
        if (team(dict_msg) == "yellow"):
            list_yellowTmprs.append(teplota(dict_msg))
            time_yellow = cas(dict_msg)
            
        
        dict_data = {"blue" : {"prumerna": statistics_(list_blueTmprs)[0],
                               "median" : statistics_(list_blueTmprs)[1],
                               "maximalni": statistics_(list_blueTmprs)[2],
                               "minimalni": statistics_(list_blueTmprs)[3],
                               "posledni": statistics_(list_blueTmprs)[4],
                               "cas": cas(dict_msg)},
                     
                     "black" : {"prumerna": statistics_(list_blackTmprs)[0],
                               "median" : statistics_(list_blackTmprs)[1],
                               "maximalni": statistics_(list_blackTmprs)[2],
                               "minimalni": statistics_(list_blackTmprs)[3],
                               "posledni":  statistics_(list_blackTmprs)[4],
                               "cas": cas(dict_msg)},
                     
                     "pink" : {"prumerna": statistics_(list_pinkTmprs)[0],
                               "median" : statistics_(list_pinkTmprs)[1],
                               "maximalni": statistics_(list_pinkTmprs)[2],
                               "minimalni": statistics_(list_pinkTmprs)[3],
                               "posledni":  statistics_(list_pinkTmprs)[4],
                               "cas": cas(dict_msg)},
                     
                     "red" : {"prumerna": statistics_(list_redTmprs)[0],
                               "median" : statistics_(list_redTmprs)[1],
                               "maximalni": statistics_(list_redTmprs)[2],
                               "minimalni": statistics_(list_redTmprs)[3],
                               "posledni":  statistics_(list_redTmprs)[4],
                               "cas": cas(dict_msg)},
                     
                    "orange" : {"prumerna": statistics_(list_orangeTmprs)[0],
                               "median" : statistics_(list_orangeTmprs)[1],
                               "maximalni": statistics_(list_orangeTmprs)[2],
                               "minimalni": statistics_(list_orangeTmprs)[3],
                               "posledni":  statistics_(list_orangeTmprs)[4],
                               "cas": cas(dict_msg)},
                     
                      "yellow" : {"prumerna": statistics_(list_yellowTmprs)[0],
                               "median" : statistics_(list_yellowTmprs)[1],
                               "maximalni": statistics_(list_yellowTmprs)[2],
                               "minimalni": statistics_(list_yellowTmprs)[3],
                               "posledni":  statistics_(list_yellowTmprs)[3],
                               "cas": cas(dict_msg)}}
       # json_data = ast.literal_eval(json_data)
        json_file = json.dumps(dict_data, indent = 4)
        with open("fileJSON.json", 'w') as f:    
            f.write(json_file)   
            f.close()  
                                                                          
    except ValueError:
        print('Incorrect data')
   
    
def team(dict):
    team_name = dict['team_name']
    return team_name

def cas(dict):
    cas = dict['created_on'].replace("\'", "\"")
    return cas
'''
def getCorrectData(str):
    list_str = re.sub('{','',str)
    list_str =  re.sub('{','',list_str)
    list_str =  re.sub('}','',list_str)
    list_str =  re.sub(':','',list_str)
    list_str =  re.sub(',','',list_str)
    list_old = list_str.split(" ")
    list_new = ["'source'" , "'fake'" , "'team_name'", list_old[3], "'created_on'", list_old[5], "'temperature'", list_old[7]]
    str_new = "b"+"\'"+"{"+list_new[0]+": "+list_new[1]+", "+list_new[2]+": "+list_new[3]+", "+list_new[4]+": "+list_new[5]+","+list_new[6]+": "+list_new[7]+"}"+"\'"
    return str_new
'''
def main():
    client = mqtt.Client()
    client.on_connect = on_connect
    client.on_message = on_message

    client.username_pw_set('mqtt_student', password='pivo')

    client.connect(SERVER, 1883, 60)

    # Blocking call that processes network traffic, dispatches callbacks and
    # handles reconnecting.
    # Other loop*() functions are available that give a threaded interface and
    # a manual interface.
    client.loop_forever()


if __name__ == '__main__':
    main()

Connected with result code qos: 0
b'{"source": "fake", "team_name": "blue", "created_on": "2020-05-06T23:07:53.837262", "temperature": -1.2280908768666818}'
b'{"source": "fake", "team_name": "pink", "created_on": "2020-05-06T23:07:59.148359", "temperature": 8.442580650268555}'
b'{"source": "fake", "team_name": "yellow", "created_on": "2020-05-06T23:08:04.000233", "temperature": 6.277828170569196}'
b'{"source": "fake", "team_name": "black", "created_on": "2020-05-06T23:08:06.081344", "temperature": 6.2277032255716325}'
b'{"source": "fake", "team_name": "orange", "created_on": "2020-05-06T23:08:10.082780", "temperature": 7.03400826774532}'
b'{"source": "fake", "team_name": "green", "created_on": "2020-05-06T23:08:47.692046", "temperature": 2.3694065165641094}'
b'{"source": "fake", "team_name": "blue", "created_on": "2020-05-06T23:08:53.934377", "temperature": -1.1229095721350055}'
b'{"source": "fake", "team_name": "pink", "created_on": "2020-05-06T23:08:59.245843", "temperature": 8.97557